In [17]:
import numpy as np
import itertools
from scipy.linalg import solve_triangular

In [18]:
T = lambda n: np.ones((1,n))
TT = lambda n: np.ones((n,n))
I = lambda n: np.eye(n)
def proj(n):
    R = np.zeros((n,2))
    R[0,0] = 1
    R[1:,1] = 1
    return R

def marginals(domain, weights):
    result = []
    cubes = list(itertools.product(*[[1,0]]*len(domain)))
    subs = [(T(n), I(n)) for n in domain]
    for wgt, cube in zip(weights, cubes):
        sub = reduce(np.kron, [sub[i] for sub, i in zip(subs, cube)])
        result.append(wgt * sub)
    return np.vstack(result)

def idx(key):
    d = len(key)
    return np.sum([b*2**k for b, k in zip(key, range(len(key)))])

def MtM(domain, weights):
    result = []
    cubes = list(itertools.product(*[[1,0]]*len(domain)))
    subs = [(TT(n), I(n)) for n in domain]
    for cube in cubes:
        wgt = weights[idx(np.array(cube).astype(bool))]
        sub = reduce(np.kron, [sub[i] for sub, i in zip(subs, cube)])
        result.append(wgt * sub)
    return sum(result)

In [ ]:
dom = (3,4)
weights = np.random.rand(2**len(dom))
theta = np.random.rand(2**len(dom))

W = marginals(dom, weights)
A = marginals(dom, theta)

print np.trace(W.T.dot(W).dot(np.linalg.pinv(A.T.dot(A))))

In [ ]:
P = reduce(np.kron, [proj(n) for n in dom])
W1 = W.dot(P)
A1 = A.dot(P)

WtW1 = W1.T.dot(W1)
AtA1 = A1.T.dot(A1)
print WtW1.dot(np.linalg.pinv(AtA1))[0,0]*np.prod(dom)

In [ ]:
WtW1 = W1.T.dot(W1)
AtA1 = A1.T.dot(A1)
AtA1inv = np.linalg.pinv(AtA1)
print weights
print WtW1[0]
print AtA1inv[0]
print np.dot(WtW1[0], AtA1inv[0])
# Note: the first row of WtW can be expressed directly terms of the weights
print np.sum(weights**2)

err = np.linalg.lstsq(AtA1, WtW1[0])[0][0]
print err, err*np.prod(dom)

In [ ]:
theta = np.sqrt([ 0.03245202,  0.90396168,  0.24949629,  0.56501183])
AtA = MtM(dom, theta**2)

def idx(key):
    return np.sum([b*2**k for b, k in zip(key, range(len(key)))])

def inverse(dom, theta):
    # Compute the inverse of A^T A for a marginals-like strategy
    d = len(dom)
    mtables = map(np.array, itertools.product(*[[True, False]]*d))
    X = np.zeros((2**d, 2**d))
    for A in mtables:
        for B in mtables:
            i = idx(A & B)
            j = idx(A)
            k = idx(B)
            X[i,j] += theta[k] * np.prod(np.array(dom)[~(A | B)])
    z = np.zeros(2**d)
    z[-1] = 1.0
    print X
    return solve_triangular(X, z) # Ryan: do we need to reorder rows of X and z?
    #return np.linalg.solve(X, z)
    
def trace(dom, weights, c):
    d = len(dom)
    mtables = map(np.array, itertools.product(*[[True, False]]*d))
    ans = 0.0
    for A in mtables:
        for B in mtables:
            i = idx(A)
            j = idx(B)
            mult = np.prod(np.array(dom)[~(A|B)])
            ans += mult * weights[i] * c[j]
    return ans * np.prod(dom)

c = inverse(dom, theta**2)
print c
print np.linalg.norm(MtM(dom, c) - np.linalg.pinv(AtA))
print trace(dom, weights**2, c)
WtW = MtM(dom, weights**2)
print np.trace(np.linalg.pinv(AtA).dot(WtW))

In [ ]:
import autograd.numpy as np
from autograd.scipy.linalg import solve_triangular 
from autograd import grad
from autograd import primitive

dom = (3,4,5)
d = len(dom)
theta = np.random.rand(2**d)
weights = np.random.rand(2**d)
AtA = MtM(dom, theta**2)
WtW = MtM(dom, weights**2)

@primitive
def Xtheta(theta, mult):
    # Function from R^n -> R^{n x n}
    # d X[a&b,a] / d T[b] = mult[a|b]
    B = np.arange(2**d)
    Xs = [np.bincount(a&B, weights=theta*mult[a|B], minlength=2**d) for a in range(2**d)]
    return np.vstack(Xs).T

def err(dom, weights, theta):
    # E(t) = h(c(inv(X(t))))
    d = len(dom)
    mult = np.ones(2**d)
    for i in range(2**d):
        for k in range(d):
            if not (i & (2**k)):
                mult[i] *= dom[k]
                
    B = np.arange(2**d)
    Xs = [np.bincount(a&B, weights=theta*mult[a|B], minlength=2**d) for a in range(2**d)]
    X = np.vstack(Xs).T
    
    X = Xtheta(theta, mult)
    #print X.shape
                
#    X = np.zeros((2**d, 2**d))
#    for a in range(2**d):
#        for b in range(2**d):
#            X[a&b,a] += theta[b] * mult[a|b]
            
    z = np.zeros(2**d)
    z[-1] = 1.0
    inv = solve_triangular(X, z)
    
    ans = [mult[a|b]*weights[a]*inv[b] for a in range(2**d) for b in range(2**d)]
    return np.prod(dom) * np.sum(np.array(ans))

def foo(theta):
    return err(dom, weights**2, theta**2)

print np.trace(np.linalg.pinv(AtA).dot(WtW))
print err(dom, weights**2, theta**2)
print foo(theta)
#print grad(foo)(theta)

In [ ]:
v = np.random.rand(5)
def bar(x, i=0):
    Y = np.triu(np.reshape(x, (5,5)))
    return solve_triangular(Y, v)[i]

def bar_grad(x, i=0):
    X = np.triu(np.reshape(x, (5,5)))
    Y = np.linalg.inv(X)
    D = np.zeros_like(Y)
    D[i] = v
    A1 = solve_triangular(X.T, D, lower=True)
    A2 = solve_triangular(X, A1.T)
    return -np.triu(A2.T)
    

x = np.random.rand(25)
print bar(x)
print np.linalg.norm(grad(bar)(x).reshape(5,5) - bar_grad(x))

In [ ]:
from autograd import jacobian

z = np.random.rand(6)
def foo(Y):
    return np.dot(Y, z)[0]

def bar(X):
    Y = np.linalg.inv(X)
    return foo(Y)

def deriv1(Y):
    ans = np.zeros((6,6))
    ans[0] = z
    return ans

def deriv2(X):
    Y = np.linalg.inv(X)
    
    J = jacobian(np.linalg.inv)(X)
    D = deriv1(Y)
    A = np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            for k in range(6):
                for l in range(6):
                    A[i,j] += J[k,l,i,j] * D[k,l]
    return A

def deriv3(X):
    Y = np.linalg.inv(X)
    D = deriv1(Y)
    return -Y.T.dot(D).dot(Y.T)

X = np.random.rand(6,6)

print foo(X)
print deriv1(X)
#print grad(bar)(X)
print np.linalg.norm(grad(bar)(X) - deriv3(X))

In [ ]:
@primitive
def logsumexp(x):
    """Numerically stable log(sum(exp(x)))"""
    max_x = np.max(x)
    return max_x + np.log(np.sum(np.exp(x - max_x)))

def logsumexp_vjp(g, ans, vs, gvs, x):
    x_shape = x.shape
    return np.full(x_shape, g) * np.exp(x - np.full(x_shape, ans))

logsumexp.defvjp(logsumexp_vjp)

x = np.random.rand(10)
print logsumexp(x)
print grad(logsumexp)(x)

In [ ]:
from autograd import jacobian 


d = len(dom)
mult = np.ones(2**d)
for i in range(2**d):
    for k in range(d):
        if not (i & (2**k)):
            mult[i] *= dom[k]

@primitive
def Xtheta(theta):
    # Function from R^n -> R^{n x n}
    # d X[a&b,a] / d T[b] = mult[a|b]
    B = np.arange(2**d)
    Xs = [np.bincount(a&B, weights=theta*mult[a|B], minlength=2**d) for a in range(2**d)]
    return np.vstack(Xs).T

def Xtheta_vjp(g, ans, vs, gvs, theta):
    # Function from R^{n x n} -> R^n
    A = np.arange(theta.size)
    dT = [np.dot(g[A&b,A], mult[A|b]) for b in range(theta.size)]
    return np.array(dT)

Xtheta.defvjp(Xtheta_vjp)

J = jacobian(Xtheta)(theta)
X1 = Xtheta(theta)
theta[0] += 1e-6
X2 = Xtheta(theta)

for i in range(8):
    for j in range(8):
        pass
        #print jacobian(Xtheta)(theta)[i,j,0], (X2[i,j]-X1[i,j])/1e-6

In [ ]:
import autograd.numpy as np
from autograd.scipy.linalg import solve_triangular 
from autograd import grad
from autograd import primitive

dom = (3,4,5)
d = len(dom)
theta = np.random.rand(2**d)
weights = np.random.rand(2**d)
AtA = MtM(dom, theta**2)
WtW = MtM(dom, weights**2)

def err(dom, weights, theta):
    # E(t) = h(c(inv(X(t))))
    d = len(dom)
    mult = np.ones(2**d)
    for i in range(2**d):
        for k in range(d):
            if not (i & (2**k)):
                mult[i] *= dom[k]
                
    @primitive
    def Xtheta(theta):
        # Function from R^n -> R^{n x n}
        # d X[a&b,a] / d T[b] = mult[a|b]
        B = np.arange(2**d)
        Xs = [np.bincount(a&B, weights=theta*mult[a|B], minlength=theta.size) for a in range(theta.size)]
        return np.vstack(Xs).T
    
    def Xtheta_vjp(g, ans, vs, gvs, theta):
        # Function from R^{n x n} -> R^n
        A = np.arange(theta.size)
        dT = [np.dot(g[A&b,A], mult[A|b]) for b in range(theta.size)]
        return np.array(dT)
    
    Xtheta.defvjp(Xtheta_vjp)
    
    X = Xtheta(theta)
            
    z = np.zeros(2**d)
    z[-1] = 1.0
    inv = solve_triangular(X, z)
    
    ans = [mult[a|b]*weights[a]*inv[b] for a in range(2**d) for b in range(2**d)]
    return np.prod(dom) * np.sum(np.array(ans))

def foo(theta):
    return err(dom, weights**2, theta**2)

print np.trace(np.linalg.pinv(AtA).dot(WtW))
print err(dom, weights**2, theta**2)
print foo(theta)
print grad(foo)(theta)

In [3]:
import autograd.numpy as np
from autograd.scipy.linalg import solve_triangular 
from autograd import grad
from autograd import primitive
from scipy.sparse.linalg import lsmr

dom = (3,4,5,6)
d = len(dom)
theta = np.random.rand(2**d)
weights = np.random.rand(2**d)
weights[-1] = 0
theta[-1] = 0
AtA = MtM(dom, theta**2)
WtW = MtM(dom, weights**2)

def err(dom, weights, theta):
    assert weights[-1] == 0 or theta[-1] > 0
    # E(t) = h(c(inv(X(t))))
    d = len(dom)
    mult = np.ones(2**d)
    for i in range(2**d):
        for k in range(d):
            if not (i & (2**k)):
                mult[i] *= dom[k]
    @primitive
    def Xtheta(theta):
        # Function from R^n -> R^{n x n}
        # d X[a&b,a] / d T[b] = mult[a|b]
        B = np.arange(2**d)
        Xs = [np.bincount(a&B, weights=theta*mult[a|B], minlength=theta.size) for a in range(theta.size)]
        return np.vstack(Xs).T
    
    def Xtheta_vjp(g, ans, vs, gvs, theta):
        # Function from R^{n x n} -> R^n
        A = np.arange(theta.size)
        dT = [np.dot(g[A&b,A], mult[A|b]) for b in range(theta.size)]
        return np.array(dT)
    
    Xtheta.defvjp(Xtheta_vjp)
    X = Xtheta(theta)
    theta2 = np.dot(X, theta)
    X = Xtheta(theta2)
    
    Y = X + np.diag(np.diag(X) == 0) # to deal with underdetermined systems
    inv = solve_triangular(Y, theta)
    
    ans = [mult[a|b]*weights[a]*inv[b] for a in range(2**d) for b in range(2**d)]
    
    A = np.arange(2**d)
    ans = [np.dot(mult[A|b],weights*inv[b]) for b in range(2**d)]
    
    return np.prod(dom) * np.sum(np.array(ans))

def foo(theta):
    return err(dom, weights**2, theta**2)

print np.trace(np.linalg.pinv(AtA).dot(WtW))
#print err(dom, weights**2, theta**2)
print foo(theta)
print grad(foo)(theta)

NameError: name 'MtM' is not defined

In [ ]:
np.apply_along_axis(np)

In [ ]:
def tmp(theta):
    # Function from R^n -> R^{n x n}
    # d X[a&b,a] / d T[b] = mult[a|b]
    
    B = np.arange(theta.size)
    Xs = [np.bincount(a&B, weights=theta, minlength=theta.size) for a in range(2**d)]
    return np.vstack(Xs).T

for d in range(2,14):
    dense = 2.0**(2*d)
    sparse = np.sum(tmp(np.random.rand(2**d))>0)
    print d, dense, sparse, 3**d, sparse/dense

In [168]:
import autograd.numpy as np
from autograd import jacobian
from autograd.scipy.linalg import solve_triangular
from autograd import primitive

v = np.random.rand(5)
def foo(X):
    return solve_triangular(X, v)

def myjac(X, i=0):
    e = np.zeros(5)
    e[i] = 1
    v = solve_triangular(X.T, e, lower=True)
    return -np.triu(np.outer(v, foo(X)))

X = np.triu(np.random.rand(5,5))

print foo(X)
J1 = jacobian(foo)(X)
for i in range(5):
    print np.linalg.norm(J1[i,:,:] - myjac(X, i))

[ 1.1194 -4.0237  1.0553 -1.8257  1.711 ]
0.0
0.0
0.0
0.0
0.0


In [127]:
import numpy as np
from scipy.sparse.linalg import spsolve_triangular
from scipy.linalg import solve_triangular 
from scipy import sparse
import time

dom = (3,4,5)
d = len(dom)
theta = np.random.rand(2**d)
weights = np.random.rand(2**d)
AtA = MtM(dom, theta**2)
WtW = MtM(dom, weights**2)

def err(dom, weights, theta):
    assert weights[-1] == 0 or theta[-1] > 0
    # E(t) = h(c(inv(X(t))))
    d = len(dom)
    mult = np.ones(2**d)
    for i in range(2**d):
        for k in range(d):
            if not (i & (2**k)):
                mult[i] *= dom[k]
              
    # Dense
    B = np.arange(2**d)
    #Xs = [np.bincount(a&B, theta*mult[a|B], theta.size) for a in range(theta.size)]
    #X = np.vstack(Xs).T
    
    # Sparse
    values = np.zeros(3**d)
    rows = np.zeros(3**d, dtype=int)
    cols = np.zeros(3**d, dtype=int)
    start = 0
    for a in range(2**d):
        #uniq, rev = np.unique(a&B, return_inverse=True) # most of time being spent here
        msk = np.zeros(2**d, dtype=int)
        msk[B&a] = 1
        uniq = np.nonzero(msk)[0]
        step = uniq.size
        msk[uniq] = np.arange(step)
        rev = msk[B&a]
        values[start:start+step] = np.bincount(rev, theta*mult[a|B], step)
        cols[start:start+step] = a
        rows[start:start+step] = uniq
        start += step
        
    X = sparse.csr_matrix((values, (rows, cols)), (2**d, 2**d))
    
    z = np.zeros(2**d)
    z[-1] = 1.0
    inv = spsolve_triangular(X, z, lower=False)
    
    A = np.arange(2**d)
    ans = [np.dot(mult[A|b],weights*inv[b]) for b in range(2**d)]
    
    return np.prod(dom) * np.sum(ans)

def foo(d):
    dom = [10]*d
    theta = np.random.rand(2**d)
    weights = np.random.rand(2**d)
    return err(dom, weights**2, theta**2)

%time foo(15)

CPU times: user 21 s, sys: 80 ms, total: 21 s
Wall time: 21 s


21253180194575180.0

In [237]:
import numpy as np
from scipy.sparse.linalg import spsolve_triangular
from scipy.linalg import solve_triangular 
from scipy import sparse
import time

dom = [10]*15
d = len(dom)
mult = np.ones(2**d)
for i in range(2**d):
    for k in range(d):
        if not (i & (2**k)):
            mult[i] *= dom[k]
weights = np.random.rand(2**d)
weights[-1] = 0
A = np.arange(2**d)

def f(phi):
    n = np.prod(dom)
    # Note: not mulitplying by n
    ans = np.sum([phi[b]*np.dot(mult[A|b],weights**2) for b in range(2**d)])
    dphi = np.array([np.dot(weights**2, mult[A|b]) for b in range(2**d)])
    return ans, dphi

def Xmatrix(theta2):
    values = np.zeros(3**d)
    rows = np.zeros(3**d, dtype=int)
    cols = np.zeros(3**d, dtype=int)
    start = 0
    for b in range(2**d):
        #uniq, rev = np.unique(a&B, return_inverse=True) # most of time being spent here
        msk = np.zeros(2**d, dtype=int)
        msk[A&b] = 1
        uniq = np.nonzero(msk)[0]
        step = uniq.size
        msk[uniq] = np.arange(step)
        rev = msk[A&b]
        #print start, start+step, rev
        values[start:start+step] = np.bincount(rev, theta2*mult[A|b], step)
        cols[start:start+step] = b
        rows[start:start+step] = uniq
        start += step
    X = sparse.csr_matrix((values, (rows, cols)), (2**d, 2**d))
    # Note: If X is not full rank, need to modify it so that solve_triangular works
    # This doesn't impact the gradient calculations though
    # a finite difference sanity check might suggest otherwise, 
    # but a valid subgradient at theta_k = 0 is 0 due to symmetry
    D = sparse.diags((X.diagonal()==0).astype(np.float64))
    return X+D

def E(theta):
    theta2 = theta**2
    Y = Xmatrix(theta2)
    params = Y.dot(theta2)
    X = Xmatrix(params)
    phi = spsolve_triangular(X, theta2, lower=False)
    ans, dphi = f(phi)
    dXvect = -spsolve_triangular(X.T, dphi, lower=True)
    # dX = outer(dXvect, phi)
    dparams = np.array([np.dot(dXvect[A&b]*phi, mult[A|b]) for b in range(2**d)])
    dtheta2 = Y.T.dot(dparams)
    return ans, 2*theta*dtheta2

def check_E():
    eps = 1e-6
    theta = np.random.rand(2**d)
    #theta[-1] = 0
    ans, dtheta = E(theta)
    print dtheta
    approx = np.zeros(2**d)
    for i in range(2**d):
        theta[i] += eps
        tmp, _ = E(theta)
        approx[i] = (tmp - ans) / eps
        theta[i] -= eps
    print approx
    
theta = np.random.rand(2**d)
#AtA = MtM(dom, theta**2)
#WtW = MtM(dom, weights**2)
#print np.trace(WtW.dot(np.linalg.pinv(AtA)))
%time print E(theta)[0]
#check_E()

5.15716769771
CPU times: user 48.8 s, sys: 140 ms, total: 49 s
Wall time: 49 s


In [197]:
theta = np.random.rand(2**d)
theta[-1] = 0
X = Xmatrix(theta)
sparse.diags((X.diagonal()==0).astype(float)).toarray()

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [175]:
import numpy as np
from scipy.sparse.linalg import LinearOperator, aslinearoperator
def MtM_linop(domain, weights):
    d = len(domain)
    n = np.prod(domain)
    def axes(i):
        return tuple([k for k in range(d) if not i & 2**k])
    zipped = [(weights[i], axes(i)) for i in range(2**d) if weights[i] != 0]
    def matvec(v):
        X = v.reshape(domain)
        Y = np.zeros(domain, dtype=weights.dtype)
        for w, ax in zipped:
            Y += w*X.sum(axis=ax, keepdims=True)
        return Y.flatten()
    return LinearOperator((n,n), matvec, matvec, dtype=weights.dtype)

def M_linop(domain, weights):
    d = len(domain)
    def axes(i):
        return tuple([k for k in range(d) if not i & 2**k])
    def size(i):
        return np.prod([domain[k] for k in range(d) if i & 2**k], dtype=int)
    zipped = [(weights[i], axes(i)) for i in range(2**d) if weights[i] != 0]
    sizes = [size(i) for i in range(2**d) if weights[i] != 0]
    idx = np.cumsum(sizes)[:-1]
    def matvec(x):
        X = x.reshape(domain)
        return np.concatenate([w*X.sum(axis=ax).flatten() for w,ax in zipped])
    def rmatvec(y):
        ys = np.split(y, idx)
        X = np.zeros(domain)
        for (w,ax), z in zip(zipped, ys):
            tmp = tuple([1 if k in ax else domain[k] for k in range(d)])
            X += w*z.reshape(tmp)
        return X.flatten()
    
    m, n = sum(sizes), np.prod(domain)
    return LinearOperator((m,n), matvec, rmatvec, dtype=weights.dtype)

def M_dense(domain, weights):
    result = []
    def axes(i):
        return tuple([k for k in range(d) if not i & 2**k])
    zipped = [(weights[i], axes(i)) for i in range(2**d) if weights[i] != 0]
    base = [np.eye(n) for n in domain]
    for w, ax in zipped:
        tmp = list(base)
        for k in ax: tmp[k] = np.ones(domain[k])
        result.append(w*reduce(np.kron, tmp))
    return np.vstack(result)        

In [176]:
dom = (2,3,4)
d = len(dom)
dtype = np.float64
w = np.random.rand(2**d).astype(dtype)
#w[np.random.rand(2**d)<0.75] = 0
MtM1 = MtM_linop(dom, w)
#MtM2 = MtM(dom, w)
v = np.random.rand(np.prod(dom)).astype(dtype)
#print np.linalg.norm(MtM1.dot(v) - MtM2.dot(v))
#print np.linalg.norm(MtM1.H.dot(v) - MtM2.T.dot(v))
print 'checkpt'
%time y = MtM1.dot(v)

checkpt
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 104 µs


In [177]:
MtM1 = MtM_linop(dom, w**2)
M1 = M_linop(dom, w)
print np.linalg.norm(M1.H.dot(M1.dot(v)) - MtM1.dot(v))

M = M_dense(dom, w)

print np.linalg.norm(M.T.dot(M).dot(v) - MtM1.dot(v))
print np.linalg.norm(M.dot(v) - M1.dot(v))
u = np.random.rand(M1.shape[0])
print np.linalg.norm(M.T.dot(u) - M1.H.dot(u))

1.25607396695e-15
4.4408920985e-15
1.11731275387e-15
0.0


In [109]:
np.split(np.random.rand(10), [2,3])

[array([ 0.40490196,  0.88492224]),
 array([ 0.18752642]),
 array([ 0.02019139,  0.59657716,  0.93521953,  0.80511318,  0.72044427,
         0.3623848 ,  0.94657748])]

array([ True,  True,  True], dtype=bool)